# Question 6

## Utils

In [17]:
import numpy as np
import time

def init_data_binary(n, m):
    '''
    Generate random dataset of size n, dimension m, with the binary label 
    '''
    X=np.random.rand(n, m)
    y=np.random.rand(n, 1)
    for i in range(n):
        y[i]=-1 if y[i]<0.5 else 1 # binary conversion
    return X,y
    
def init_data_cont(n, m):
    '''
    Generate random dataset of size n, dimension m, with continuous label in [0,1]
    '''
    X=np.random.rand(n, m)
    y=np.random.rand(n, 1)
    return X,y

def init_weights(m):
    '''
    Generate random weights in [0,1]
    '''
    return np.random.rand(m, 1)

def numerical_grad(func_obj, w, epsilon):
    '''
    compute the gradient numberically and return
    '''
    m = w.shape[0] # number of features
    grad = np.zeros(m)
    for i in range(m):
        wp = np.copy(w) # positive direction ?
        wn = np.copy(w) # negative direction ?
        wp[i] = w[i] + epsilon
        wn[i] = w[i] - epsilon
        grad[i] = (func_obj(wp) - func_obj(wn)) / (2*epsilon)
    return grad

## Question 6.1

In [18]:
def inverse_sigmoid(var):
    '''
    return 1/(1+exp(var))
    avoid large overflow
    '''
    if var>=0:
        return np.exp(-var)/(1+np.exp(-var))
    else:
        return 1/(1+np.exp(var))

def logistic_loss(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.1
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0] # size of dataset
    # m = X.shape[1] # number of features
    f = 0
    g = 0
    for i in range(n):
        x_cdot_w = np.dot(X[i], w) # w^T x
        f = f + np.logaddexp(0, -y[i] * x_cdot_w)
        g = g + -y[i] * inverse_sigmoid(y[i]* x_cdot_w) * X[i] # exp(-y_iw^Tx_i)/(1+exp(-y_iw^Tx_i)) * (-y_i x_i)
    f = f + la * np.sum(np.multiply(w, w))
    g = g + 2 * la * w.reshape(1,-1)
    return f,g.reshape(-1, 1)

def logistic_loss_vectorized(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.1
    return f as the loss function value and g as the gradient
    '''
    x_mul_w = np.matmul(X, w) # n * 1
    y_X_w = np.multiply(y, x_mul_w) # n * 1
    f = np.sum(np.logaddexp(0, -y_X_w)) + la * np.sum(np.multiply(w, w))
    inv_sigmoid_vector = np.exp(np.minimum(0, -y_X_w))/(1 + np.exp(-y_X_w*np.sign(y_X_w))) # n * 1
    y_inv_sigmoid = -1 * np.multiply(y, inv_sigmoid_vector) # n * 1
    g =  np.matmul(y_inv_sigmoid.T, X) + 2 * la * w.T
    return f,g

## Test Block for Question 6.1

In [20]:
# Test Assignment 6.1
def test_assignment_6_1(n, m, la):
    '''
    test the function logistic_loss
    n: size of dataset
    m: number of features
    la: lambda
    '''
    print('''### logistic_loss simple loop ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    start=time.time()
    f,g=logistic_loss(w, X, y, la)
    end=time.time()
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f)
    print('''# gradient: ''')
    print(g.T)

def test_assignment_6_1_step7_compare(n, m, la):
    print('''### logistic_loss compare simple loop version and vectorized version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    start=time.time()
    f1, g1 = logistic_loss(w, X, y, la)
    end=time.time()
    print('''## Result of simple loop version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f1)
    print('''# gradient: ''')
    print(g1.T)
    start=time.time()
    f2, g2 = logistic_loss_vectorized(w, X, y, la)
    end=time.time()
    print('''## Result of vectorized version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f2)
    print('''# gradient: ''')
    print(g2)
    
def test_assignment_6_1_step8_compare(n, m, la, epsilon):
    print('''### logistic_loss compare gradient, derivative version and numerical version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    print('''# epsilon: '''),
    print(epsilon)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    _, g1 = logistic_loss(w, X, y, la)
    func_obj = lambda w : logistic_loss_vectorized(w, X, y, la)[0]
    g2 = numerical_grad(func_obj, w, epsilon)
    print('''## Result of derivative version ##''')
    print('''# gradient: ''')
    print(g1.T)
    print('''## Result of numerical version ##''')
    print('''# gradient: ''')
    print(g2.T)
    
n,m,la=100,10,1.0
test_assignment_6_1(n,m,la)
n,m,la=100,100000,1.0
test_assignment_6_1(n,m,la)
n,m,la=100000,10,1.0
test_assignment_6_1(n,m,la)
n,m,la=100,10,1.0
test_assignment_6_1_step7_compare(n, m, la)
n,m,la=100,100000,1.0
test_assignment_6_1_step7_compare(n, m, la)
n,m,la=100000,10,1.0
test_assignment_6_1_step7_compare(n, m, la)
n,m,la,epsilon=100,10,1.0,0.001
test_assignment_6_1_step8_compare(n, m, la, epsilon)

### logistic_loss simple loop ###
# number of cases:  100
# number of features:  10
# lambda:  1.0
# time elapsed:  0.00320196151733
# loss value:  [97.51352942]
# gradient: 
[[17.43959978 16.68665159 18.27405113 13.6839395  14.97967404 15.24908041
  16.23486807 16.60280774 14.06388353 18.66726558]]
### logistic_loss simple loop ###
# number of cases:  100
# number of features:  100000
# lambda:  1.0
# time elapsed:  0.300104141235
# loss value:  [1359088.89206763]
# gradient: 
[[25.03903844 27.41735439 25.96539733 ... 28.40512077 27.65472097
  27.0025078 ]]
### logistic_loss simple loop ###
# number of cases:  100000
# number of features:  10
# lambda:  1.0
# time elapsed:  2.14091801643
# loss value:  [114122.29526568]
# gradient: 
[[18927.09114158 19047.93958542 19371.1410727  19399.91818725
  19299.85842125 18780.58301381 18971.16171261 19105.82265923
  19320.21006099 19005.64364169]]
### logistic_loss compare simple loop version and vectorized version ###
# number of cases:  100
#

## Question 6.2 Hinge Loss/SVMs

In [21]:
def hinge_loss(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.2
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0] # size of dataset
    # m = X.shape[1] # number of features
    f = 0
    g = 0
    for i in range(n):
        one_minus_y_w_x = 1.0 - y[i]* (np.dot(X[i], w))
        if one_minus_y_w_x >=0.0:
            f = f + one_minus_y_w_x
            g = g + - y[i]*X[i]
    f = f + la * np.sum(w*w)
    g = g + 2 * la * w.reshape(1,-1)
    return f,g.reshape(-1)

def hinge_loss_vectorized(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.2
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0]
    X_cdot_w = np.matmul(X, w)
    one_minus = 1 - np.multiply(X_cdot_w, y)
    f = np.sum(np.maximum(0, one_minus)) + la * np.sum(w*w)
    minus_y = -1 * np.multiply(y, np.double(one_minus>0))
    g = np.matmul(minus_y.reshape(1,-1),X).reshape(-1,1)  + 2 * la *w.reshape(-1,1)
    g = g.reshape(-1)
    return f, g

## Test Block for Question 6.2

In [23]:
# Test Assignment 6.2
def test_assignment_6_2(n, m, la):
    '''
    test the function hinge_loss
    n: size of dataset
    m: number of features
    la: lambda
    '''
    print('''### hinge_loss simple loop ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    start=time.time()
    f,g= hinge_loss(w, X, y, la)
    end=time.time()
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f)
    print('''# gradient: ''')
    print(g)

def test_assignment_6_2_step7_compare(n, m, la):
    print('''### hinge_loss compare simple loop version and vectorized version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    start=time.time()
    f1, g1 = hinge_loss(w, X, y, la)
    end=time.time()
    print('''## Result of simple loop version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f1)
    print('''# gradient: ''')
    print(g1)
    start=time.time()
    f2, g2 = hinge_loss_vectorized(w, X, y, la)
    end=time.time()
    print('''## Result of vectorized version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f2)
    print('''# gradient: ''')
    print(g2)

def test_assignment_6_2_step8_compare(n, m, la, epsilon):
    print('''### hinge_loss compare gradient, derivative version and numerical version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    print('''# epsilon: '''),
    print(epsilon)
    X,y = init_data_binary(n, m)
    w = init_weights(m)
    _, g1 = hinge_loss_vectorized(w, X, y, la)
    func_obj = lambda w : hinge_loss_vectorized(w, X, y, la)[0]
    g2 = numerical_grad(func_obj, w, epsilon)
    print('''## Result of derivative version ##''')
    print('''# gradient: ''')
    print(g1)
    print('''## Result of numerical version ##''')
    print('''# gradient: ''')
    print(g2)

n,m,la=100,10,1.0
test_assignment_6_2(n,m,la)
n,m,la=100,100000,1.0
test_assignment_6_2(n,m,la)
n,m,la=100000,10,1.0
test_assignment_6_2(n,m,la)
n,m,la=100,10,1.0
test_assignment_6_2_step7_compare(n, m, la)
n,m,la=100,100000,1.0
test_assignment_6_2_step7_compare(n, m, la)
n,m,la=100000,10,1.0
test_assignment_6_2_step7_compare(n, m, la)
n,m,la,epsilon=100,10,1.0,0.000001
test_assignment_6_2_step8_compare(n, m, la, epsilon)

### hinge_loss simple loop ###
# number of cases:  100
# number of features:  10
# lambda:  1.0
# time elapsed:  0.0014340877533
# loss value:  [207.05022177]
# gradient: 
[26.61302737 23.65553052 28.27582656 26.89065468 24.72331345 28.44794822
 28.0209767  28.04013649 27.884543   27.84634945]
### hinge_loss simple loop ###
# number of cases:  100
# number of features:  100000
# lambda:  1.0
# time elapsed:  0.0392060279846
# loss value:  [1288272.94689123]
# gradient: 
[26.59113676 23.43390758 26.09910254 ... 24.54239136 26.58656263
 25.13508917]
### hinge_loss simple loop ###
# number of cases:  100000
# number of features:  10
# lambda:  1.0
# time elapsed:  0.805207967758
# loss value:  [220874.88007107]
# gradient: 
[25080.48788615 25022.4558949  25166.71296261 24985.14973774
 25030.54142605 25162.7541822  25176.5215486  25111.73891757
 25135.1852438  25185.4551702 ]
### hinge_loss compare simple loop version and vectorized version ###
# number of cases:  100
# number of features:

## Question 6.3 Simple Two Layers Loss

In [24]:
def simple_two_layers_loss(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.3
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0] # size of dataset
    # m = X.shape[1] # number of features
    f = 0
    g = 0
    for i in range(n):
        x_cdot_w = np.dot(X[i], w)
        if x_cdot_w >0.0:
            f = f + np.square(y[i]-x_cdot_w)
            g = g + 2 * (y[i] - x_cdot_w) * -X[i]
        else:
            f = f + np.square(y[i])
    f = f + la * np.sum(np.multiply(w, w))
    g = g + 2 * la * w.reshape(1,-1)
    return f,g

def simple_two_layers_loss_vectorized(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.3
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0]
    m = w.shape[0]
    X_cdot_w = np.matmul(X, w) # n * 1
    X_cdot_w = X_cdot_w.reshape(-1 ,1)
    zeros = np.zeros(n) 
    zeros = zeros.reshape(-1,1) # n * 1
    X_cdot_w = np.append(X_cdot_w, zeros, axis=1)
    X_cdot_w = np.max(X_cdot_w, 1).reshape(-1, 1)
    f = np.sum(np.square( y - X_cdot_w)) + la * np.sum(np.multiply(w, w))
    aux  = np.double(X_cdot_w>0)
    minus_X = -2 * X * np.repeat(aux.reshape(-1, 1), repeats=m, axis=1)
    g = np.matmul(minus_X.transpose(), (y - X_cdot_w).reshape(-1, 1)) + 2 * la * w.reshape(-1, 1)
    g = g.reshape(1, -1)
    return f, g
    

## Test Block for Question 6.3

In [25]:
# Test Assignment 6.3
def test_assignment_6_3(n, m, la):
    '''
    test the function simple_two_layers_loss
    n: size of dataset
    m: number of features
    la: lambda
    '''
    print('''### simple_two_layers_loss simple loop ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    start=time.time()
    f,g= simple_two_layers_loss(w, X, y, la)
    end=time.time()
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f)
    print('''# gradient: ''')
    print(g)

def test_assignment_6_3_step7_compare(n, m, la):
    print('''### simple_two_layers_loss compare simple loop version and vectorized version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    start=time.time()
    f1, g1 = simple_two_layers_loss(w, X, y, la)
    end=time.time()
    print('''## Result of simple loop version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f1)
    print('''# gradient: ''')
    print(g1)
    start=time.time()
    f2, g2 = simple_two_layers_loss_vectorized(w, X, y, la)
    end=time.time()
    print('''## Result of vectorized version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f2)
    print('''# gradient: ''')
    print(g2)

def test_assignment_6_3_step8_compare(n, m, la, epsilon):
    print('''### simple_two_layers_loss compare gradient, derivative version and numerical version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    print('''# epsilon: '''),
    print(epsilon)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    _, g1 = simple_two_layers_loss_vectorized(w, X, y, la)
    func_obj = lambda w : simple_two_layers_loss_vectorized(w, X, y, la)[0]
    g2 = numerical_grad(func_obj, w, epsilon)
    print('''## Result of derivative version ##''')
    print('''# gradient: ''')
    print(g1)
    print('''## Result of numerical version ##''')
    print('''# gradient: ''')
    print(g2)
    
n,m,la=100,10,1.0
test_assignment_6_3(n,m,la)
n,m,la=100,100000,1.0
test_assignment_6_3(n,m,la)
n,m,la=100000,10,1.0
test_assignment_6_3(n,m,la)
n,m,la=100,10,1.0
test_assignment_6_3_step7_compare(n, m, la)
n,m,la=100,100000,1.0
test_assignment_6_3_step7_compare(n, m, la)
n,m,la=100000,10,1.0
test_assignment_6_3_step7_compare(n, m, la)
n,m,la,epsilon=100,10,1.0,0.000001
test_assignment_6_3_step8_compare(n, m, la, epsilon)

### simple_two_layers_loss simple loop ###
# number of cases:  100
# number of features:  10
# lambda:  1.0
# time elapsed:  0.0022120475769
# loss value:  [231.18773541]
# gradient: 
[[151.70479005 129.37382117 125.30059159 155.80508924 145.02955707
  140.34168058 143.09365479 159.14077117 135.72555122 142.10143867]]
### simple_two_layers_loss simple loop ###
# number of cases:  100
# number of features:  100000
# lambda:  1.0
# time elapsed:  0.125385046005
# loss value:  [6.24354448e+10]
# gradient: 
[[2461679.10200254 2524508.02920337 2657493.45487811 ... 2620020.86495628
  2474358.23012109 2586716.95592618]]
### simple_two_layers_loss simple loop ###
# number of cases:  100000
# number of features:  10
# lambda:  1.0
# time elapsed:  1.33608102798
# loss value:  [447172.81115658]
# gradient: 
[[203214.3677658  214580.33629863 215874.04893315 206489.74855917
  205566.17282891 218493.60304622 207852.30486409 206167.13290833
  212487.02281908 215937.28636247]]
### simple_two_layers_l

## Question 6.4 Least Square Loss

In [26]:
def least_square_loss(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.4
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0] # size of dataset
    # m = X.shape[1] # number of features
    f = 0
    g = 0
    for i in range(n):
        x_cdot_w = np.dot(X[i], w)
        f = f + np.square(y[i]-x_cdot_w)
        g = g + 2 * (y[i] - x_cdot_w) * -X[i]
    f = f + la * np.sum(w*w)
    g = g + 2 * la * w.reshape(1,-1)
    return f,g

def least_square_loss_vectorized(w, X, y, la):
    '''
    input: w parameter, X variables, y label, la lambda
    Assignment 6.4
    return f as the loss function value and g as the gradient
    '''
    n = X.shape[0]
    m = w.shape[0]
    X_cdot_w = np.matmul(X, w) # n * 1
    f = np.sum(np.square( y - X_cdot_w)) + la * np.sum(np.multiply(w, w))
    minus_X = -2 * X # n * m
    g = np.matmul(minus_X.transpose(), y - X_cdot_w) + 2 * la * w.reshape(-1, 1)
    g = g.reshape(1, -1)
    return f, g
    

## Test Block for Question 6.4

In [27]:
# Test Assignment 6.4
def test_assignment_6_4(n, m, la):
    '''
    test the function least_square_loss
    n: size of dataset
    m: number of features
    la: lambda
    '''
    print('''### least_square_loss simple loop ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    start=time.time()
    f,g= least_square_loss(w, X, y, la)
    end = time.time()
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f)
    print('''# gradient: ''')
    print(g)

def test_assignment_6_4_step7_compare(n, m, la):
    print('''### least_square_loss compare simple loop version and vectorized version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    start=time.time()
    f1, g1 = least_square_loss(w, X, y, la)
    end = time.time()
    print('''## Result of simple loop version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f1)
    print('''# gradient: ''')
    print(g1)
    start=time.time()
    f2, g2 = least_square_loss_vectorized(w, X, y, la)
    end = time.time()
    print('''## Result of vectorized version ##''')
    print('''# time elapsed: '''),
    print(end-start)
    print('''# loss value: '''),
    print(f2)
    print('''# gradient: ''')
    print(g2)

def test_assignment_6_4_step8_compare(n, m, la, epsilon):
    print('''### least_square_loss compare gradient, derivative version and numerical version ###''')
    print('''# number of cases: '''),
    print(n)
    print('''# number of features: '''),
    print(m)
    print('''# lambda: '''),
    print(la)
    print('''# epsilon: '''),
    print(epsilon)
    X,y = init_data_cont(n, m)
    w = init_weights(m)
    _, g1 = least_square_loss(w, X, y, la)
    func_obj = lambda w : least_square_loss_vectorized(w, X, y, la)[0]
    g2 = numerical_grad(func_obj, w, epsilon)
    print('''## Result of derivative version ##''')
    print('''# gradient: ''')
    print(g1)
    print('''## Result of numerical version ##''')
    print('''# gradient: ''')
    print(g2)

n,m,la=100,10,1.0
test_assignment_6_4(n,m,la)
n,m,la=100,100000,1.0
test_assignment_6_4(n,m,la)
n,m,la=100000,10,1.0
test_assignment_6_4(n,m,la)
n,m,la=100,10,1.0
test_assignment_6_4_step7_compare(n, m, la)
n,m,la=100,100000,1.0
test_assignment_6_4_step7_compare(n, m, la)
n,m,la=100000,10,1.0
test_assignment_6_4_step7_compare(n, m, la)
n,m,la,epsilon=100,10,1.0,0.000001
test_assignment_6_4_step8_compare(n, m, la, epsilon)

### least_square_loss simple loop ###
# number of cases:  100
# number of features:  10
# lambda:  1.0
# time elapsed:  0.0018789768219
# loss value:  [646.97075866]
# gradient: 
[[246.86514408 287.05028885 264.53088889 261.09061765 229.56289477
  262.91284511 260.84003537 248.3841903  270.04947015 279.32944223]]
### least_square_loss simple loop ###
# number of cases:  100
# number of features:  100000
# lambda:  1.0
# time elapsed:  0.10099196434
# loss value:  [6.23971312e+10]
# gradient: 
[[2740576.11937244 2498666.95522744 2256001.2267427  ... 2181312.05481989
  2248624.94143555 2358738.22350247]]
### least_square_loss simple loop ###
# number of cases:  100000
# number of features:  10
# lambda:  1.0
# time elapsed:  1.20673894882
# loss value:  [436928.62112624]
# gradient: 
[[213311.13550738 216494.17483098 215687.00251062 199934.66876579
  200100.0786276  205478.25046642 202318.58957454 206253.15914215
  210255.36541307 208194.54140104]]
### least_square_loss compare simple lo